In [ ]:
import sys
from pathlib import Path

root_dir = Path.cwd().parents[1]
if not root_dir in sys.path:
    sys.path.insert(0, str(root_dir))

# import pandas as pd
import numpy as np
from scipy.ndimage import gaussian_filter1d
from scipy.signal import find_peaks, peak_widths

from placefield_detection.utils import get_spikeNr

In [ ]:
def build_struct_PC_results(nCells, nbin, trial_ct, nStats=5):
    results = {}
    results["status"] = {}

    for key in [
        "MI_value",
        "MI_p_value",
        "MI_z_score",
        "Isec_value",
        "Isec_p_value",
        "Isec_z_score",
        "SNR",
        "r_value",
    ]:
        results["status"][key] = np.full(nCells, np.NaN)

    results["fields"] = {
        "parameter": np.full(
            (nCells, 5, 4, nStats), np.NaN
        ),  ### (mean,std,CI_low,CI_top)
        "p_x": np.zeros((nCells, 5, nbin)),  ##sp.sparse.COO((nCells,3,nbin)),#
        "posterior_mass": np.zeros((nCells, 5)) * np.NaN,
        "reliability": np.zeros((nCells, 5)) * np.NaN,
        "Bayes_factor": np.zeros((nCells, 5, 2)) * np.NaN,
        "nModes": np.zeros(nCells).astype("int"),
    }

    results["firingstats"] = {
        "rate": np.full(nCells, np.NaN),
        "map": np.zeros((nCells, nbin)) * np.NaN,
        "std": np.zeros((nCells, nbin)) * np.NaN,
        "CI": np.zeros((nCells, 2, nbin)) * np.NaN,
        "trial_map": np.zeros((nCells, trial_ct, nbin)) * np.NaN,
        "trial_field": np.zeros((nCells, 5, trial_ct), "bool"),
        "parNoise": np.zeros((nCells, 2)) * np.NaN,
    }

    ## if method is called for nCells = 1, collapse data from first dimension
    for field in results.keys():
        for key in results[field].keys():
            results[field][key] = np.squeeze(results[field][key])

    return results

In [ ]:
def PF_thresholding(firing_rate_map, threshold_factor=4, sigma=4):

    place_fields = {}
    for neuron_idx, neuron_firing_map in enumerate(firing_rate_map):

        smooth_map = gaussian_filter1d(neuron_firing_map, sigma=sigma, mode="wrap")
        baseline = np.percentile(smooth_map, 50)
        _, _, std_rate = get_spikeNr(smooth_map)
        threshold = baseline + (threshold_factor * std_rate)

        # shifting the peak to center
        peak_index = np.argmax(smooth_map)
        shift_amount = len(smooth_map) // 2 - peak_index
        shifted_map = np.roll(smooth_map, shift_amount)

        field_loc, _ = find_peaks(shifted_map, height=threshold, distance=10, width=2)
        field_loc = field_loc.tolist()

        PF_amplitude = []
        place_field_width = []
        centroids = []

        if field_loc:
            PF_amplitude = shifted_map[field_loc].tolist()

            widths, _, left_ips, right_ips = peak_widths(
                shifted_map, field_loc, rel_height=0.6
            )
            place_field_width = [int(round(width)) for width in widths]

            # calculating centroids for each place field
            for i, loc in enumerate(field_loc):

                left_idx = int(np.floor(left_ips[i]))
                right_idx = int(np.ceil(right_ips[i]))

                field_indices = np.arange(left_idx, right_idx + 1)
                field_activities = shifted_map[field_indices]

                # computing center of mass
                centroid = np.sum(field_indices * field_activities) / np.sum(
                    field_activities
                )
                centroid = (centroid - shift_amount) % len(smooth_map)
                centroids.append(centroid)

        place_fields[neuron_idx] = {
            "baseline": baseline,
            "amplitude": PF_amplitude,
            "field_location": centroids,
            "place_field_width": place_field_width,
            "nbr_PF": len(centroids),
        }
    return place_fields

In [ ]:
def store_PC_results(results, firing_rate_map, nCells, threshold_factor=4, sigma=4):

    # calculating place fields for all neurons
    place_fields = PF_thresholding(firing_rate_map, threshold_factor, sigma)

    for neuron_ind in range(nCells):
        neuron_data = place_fields[neuron_ind]

        results["fields"]["nModes"][neuron_ind] = neuron_data["nbr_PF"]
        for field_ind in range(min(neuron_data["nbr_PF"], 5)):

            results["fields"]["parameter"][neuron_ind, field_ind, :, 0] = [
                neuron_data["baseline"],
                neuron_data["amplitude"][field_ind],
                neuron_data["field_location"][field_ind],
                neuron_data["place_field_width"][field_ind],
            ]

    return results

In [ ]:
filepath = ".../data/AlzheimerMice_Hayashi/579ad/Session10/PC_fields.pkl"
PC_fields = pd.read_pickle(filepath)
field_map = PC_fields["firingstats"]["map"]

In [8]:
nCells = len(field_map)
nbin = 100
trial_ct = 21

In [ ]:
results = build_struct_PC_results(nCells, nbin, trial_ct)
PC_results = store_PC_results(results, field_map, nCells)